In [1]:
import pulp


In [2]:
model_1_a = pulp.LpProblem("Manufacturing_company", pulp.LpMinimize)
products_list = ["A", "B", "C"]
months_list = ["January", "February", "March", "April"]
x_var = pulp.LpVariable.dicts(f"Monthly_production", [(i,j) for i in products_list for j in months_list], lowBound=0, cat=pulp.LpInteger)
# Time constraints
hours_per_unit = {"A":1.2, "B":1.3, "C":1}
monthly_available_hours = [500, 450, 400, 350]
for i in range(len(months_list)):
    model_1_a += hours_per_unit["A"] * pulp.lpSum(x_var["A", months_list[i]]) \
                + hours_per_unit["B"] * pulp.lpSum(x_var["B", months_list[i]]) \
                + hours_per_unit["C"] * pulp.lpSum(x_var["C", months_list[i]]) \
        <= monthly_available_hours[i], f"Avaiable_time_in_{months_list[i]}"
    
# Demand constraints
monthly_demand_matrix = [
    [100, 110, 120, 130],# demand of A in four months
    [80, 90, 100, 110],
    [120, 130, 140, 150]
]

# Meet Monthly demand
Inventory_fromlastmonth = {"A":0, "B":0, "C":0} # Initial Stage
holding_cost = 0
for month_ in range(len(months_list)):
    for i in range(len(products_list)):
        model_1_a += x_var[products_list[i], months_list[month_]] + Inventory_fromlastmonth[products_list[i]]\
            >= monthly_demand_matrix[i][month_], f"Demand for {products_list[i]} in {months_list[month_]}"
        monthly_surplus = x_var[products_list[i], months_list[month_]] + Inventory_fromlastmonth[products_list[i]] - monthly_demand_matrix[i][month_]
        Inventory_fromlastmonth[products_list[i]] = monthly_surplus
    holding_cost += 1*Inventory_fromlastmonth["A"] + 2*Inventory_fromlastmonth["B"] + 0.5*Inventory_fromlastmonth["C"]

# Overall production cost
months_list_exclued_March = ["January", "February", "April"]
# Increased production cost of B in March 
ProductionCost = 20*pulp.lpSum(x_var["A", month_] for month_ in months_list)\
                + 30*pulp.lpSum(x_var["B", month_] for month_ in months_list_exclued_March)\
                + 35*x_var["B", "March"]\
                + 10*pulp.lpSum(x_var["C", month_] for month_ in months_list)
# Objective
model_1_a += ProductionCost + holding_cost, "Objective"

model_1_a.solve()
print("Model Status:{}".format(pulp.LpStatus[model_1_a.status]))
for v in model_1_a.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_1_a.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/975093c468d749f9ba529df3c52638b9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/975093c468d749f9ba529df3c52638b9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 100 RHS
At line 117 BOUNDS
At line 130 ENDATA
Problem MODEL has 16 rows, 12 columns and 42 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29829 - 0.00 seconds
Cgl0004I processed model has 13 rows, 12 columns (12 integer (0 of which binary)) and 39 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 29829 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
C